# Circular, Array-backed Queue

## Overview

For this assignment you will implement a circular, array-backed queue data structure.

In the following class, which you are to complete, the backing array will be created and populated with `None`s in the `__init__` method, and the `head` and `tail` indexes set to sentinel values (you shouldn't need to modify `__init__`). Enqueueing and Dequeueing items will take place at the tail and head, with `tail` and `head` tracking the position of the most recently enqueued item and that of the next item to dequeue, respectively. To simplify testing, your implementation should make sure that when dequeuing an item its slot in the array is reset to `None`, and when the queue is emptied its `head` and `tail` attributes should be set to `-1`.

Because of the fixed size backing array, the `enqueue` operation is defined to raise a `RuntimeError` when the queue is full — the same exception should be raised when `dequeue` is called on an empty queue.

Finally, the `resize` method will allow the array underlying the queue to be increased in size. It is up to you how to implement this (you can either leave the elements in their current positions, though this may require "unwrapping" elements, or you can simply move all elements towards the front of the array). You may assume that `resize` will only be called with a value greater than the current length of the underlying array.

In [214]:
debugging = False

class Queue:
    def __init__(self, limit=10):
        self.data = [None] * limit
        self.head = -1
        self.tail = -1

    def enqueue(self, val):
        if self.empty():
            if debugging: print("vv initializing")
            head = 0
            self.tail = 0
        else:
            if self.head == len(self.data) - 1:
                if debugging: print("vv wrapping head")
                head = 0
            else:
                head = self.head + 1

            if head == self.tail:
                raise RuntimeError("Queue full!")

        self.head = head
        self.data[self.head] = val

    def dequeue(self):
        if self.empty():
            raise RuntimeError("Queue empty!")

        val, self.data[self.tail] = self.data[self.tail], None

        if self.head == self.tail:
            if debugging: print("vv emptying queue")
            self.head = -1
            self.tail = -1
        else:
            if self.tail == len(self.data) - 1:
                if debugging: print("vv wrapping tail")
                self.tail = 0
            else:
                self.tail += 1

        return val

    def resize(self, newsize):
        assert(len(self.data) < newsize)

        newdata = [None] * newsize
        if not self.empty():
            highest = -1
            for (i, v) in enumerate(self):
                newdata[i] = v
                highest = i

            self.tail = 0
            self.head = highest

        self.data = newdata

    def empty(self):
        return self.head == -1 and self.tail == -1

    def __bool__(self):
        return not self.empty()

    def __str__(self):
        strs = []
        for (i, v) in enumerate(self.data):
            s = '.' if v == None else str(v)
            s = '<'+s if i == self.tail else ' '+s
            s = s+'>' if i == self.head else s+' '
            strs.append(s)

        return '['+str(self.tail)[0]+':'+str(self.head)[0]+' '+ ''.join(strs) +']'

    def __repr__(self):
        return str(self)

    def __iter__(self):
        if self.empty():
            raise StopIteration()

        i = self.tail
        while i != self.head:
            yield self.data[i]

            if i < len(self.data) - 1:
                i += 1
            else:
                i = 0

        yield self.data[self.head]

In [215]:
from unittest import TestCase
tc = TestCase()

q = Queue(5)
tc.assertEqual(q.data, [None] * 5)

for i in range(5):
    q.enqueue(i)
    
with tc.assertRaises(RuntimeError):
    q.enqueue(5)

for i in range(5):
    tc.assertEqual(q.dequeue(), i)
    
tc.assertTrue(q.empty())

In [216]:
from unittest import TestCase
tc = TestCase()

q = Queue(10)

for i in range(6):
    q.enqueue(i)
    
tc.assertEqual(q.data.count(None), 4)

for i in range(5):
    q.dequeue()
    
tc.assertFalse(q.empty())
tc.assertEqual(q.data.count(None), 9)
tc.assertEqual(q.head, q.tail)
tc.assertEqual(q.head, 5)

for i in range(9):
    q.enqueue(i)

with tc.assertRaises(RuntimeError):
    q.enqueue(10)

for x, y in zip(q, [5] + list(range(9))):
    tc.assertEqual(x, y)
    
tc.assertEqual(q.dequeue(), 5)
for i in range(9):
    tc.assertEqual(q.dequeue(), i)

tc.assertTrue(q.empty())

In [217]:
from unittest import TestCase
tc = TestCase()

q = Queue(5)
for i in range(5):
    q.enqueue(i)
for i in range(4):
    q.dequeue()
for i in range(5, 9):
    q.enqueue(i)
    
with tc.assertRaises(RuntimeError):
    q.enqueue(10)

q.resize(10)

for x, y in zip(q, range(4, 9)):
    tc.assertEqual(x, y)
    
for i in range(9, 14):
    q.enqueue(i)

for i in range(4, 14):
    tc.assertEqual(q.dequeue(), i)
    
tc.assertTrue(q.empty())
tc.assertEqual(q.head, -1)